In [1]:
import altair as alt
import numpy as np
import pandas as pd
import pytz

import utils
import meet_indicators
import default_parameters

from datetime import date, datetime, timedelta
from IPython.display import Markdown
import IPython.display

# Default parameters
county_state_name = default_parameters.county_state_name
state_name = default_parameters.state_name
msa_name = default_parameters.msa_name
time_zone = default_parameters.time_zone

fulldate_format = default_parameters.fulldate_format
monthdate_format = default_parameters.monthdate_format
start_date = default_parameters.start_date
#yesterday_date = default_parameters.yesterday_date
yesterday_date = default_parameters.two_days_ago
today_date = default_parameters.today_date
two_weeks_ago = default_parameters.two_weeks_ago
two_days_ago = default_parameters.two_days_ago

# Daily testing upper and lower bound
county_test_lower_bound = 15_000
county_test_upper_bound = 16_667
city_test_lower_bound = round(county_test_lower_bound * 0.5)
city_test_upper_bound = round(county_test_upper_bound * 0.5)
positive_lower_bound = 0.04
positive_upper_bound = 0.08
hospital_bound = 0.30

In [2]:
county_case_indicator = meet_indicators.meet_case("county", county_state_name, start_date)
county_death_indicator = meet_indicators.meet_death("county", county_state_name, start_date)
county_test_indicator = meet_indicators.meet_daily_testing(yesterday_date, "county", 
                                                           county_test_lower_bound, county_test_upper_bound)
county_positive_indicator = meet_indicators.meet_positive_share(yesterday_date, "county", 
                                                                positive_lower_bound, positive_upper_bound)
acute_indicator = meet_indicators.meet_acute(yesterday_date)
icu_indicator = meet_indicators.meet_icu(yesterday_date)
ventilator_indicator = meet_indicators.meet_ventilator(yesterday_date)


city_case_indicator = meet_indicators.meet_lacity_case(start_date)
city_test_indicator = meet_indicators.meet_daily_testing(yesterday_date, "city", 
                                                               city_test_lower_bound, city_test_upper_bound)
city_positive_indicator = meet_indicators.meet_positive_share(yesterday_date, "city", 
                                                              positive_lower_bound, positive_upper_bound)

## Works - but displays at 2 tables

In [3]:
# Check cases according to some criterion
def check_cases(row):
    status = ["failed" if x < 14 else "met" if x >= 14 else "" for x in row]
    return pd.Series(status, index=row.index, dtype="category")

def check_deaths(row):
    status = ["failed" if x < 14 else "met" if x >= 14 else "" for x in row]
    return pd.Series(status, index=row.index, dtype="category")

 
def check_tests(lower_bound, upper_bound, row):
    status = ["failed" if x < lower_bound 
              else "met" if ((x >= lower_bound) and (x < upper_bound)) 
              else "exceeded" if x >= upper_bound 
              else "" for x in row]
    return pd.Series(status, index=row.index, dtype="category")

def check_positives(row):
    status = ["failed" if x > positive_upper_bound 
              else "met" if ((x > positive_lower_bound) and (x <= positive_upper_bound)) 
              else "exceeded" if x >= positive_lower_bound 
              else "" for x in row]
    return pd.Series(status, index=row.index, dtype="category")


def check_hospitals(row):
    status = ["failed" if x < hospital_bound 
              else "met" if x >= hospital_bound 
              else "" for x in row]
    return pd.Series(status, index=row.index, dtype="category")    


county_fnmap = {
    "Cases": check_cases,
    "Deaths": check_deaths,
    "Daily Testing": lambda row: check_tests(county_test_lower_bound, county_test_upper_bound, row),
    "Positive Tests": check_positives,
    "Acute Beds": check_hospitals,
    "ICU Beds": check_hospitals,
    "Ventilators": check_hospitals,
}


city_fnmap = {
    "Cases": check_cases,
    "Deaths": check_deaths,
    "Daily Testing": lambda row: check_tests(city_test_lower_bound, city_test_upper_bound, row),
    "Positive Tests": check_positives,
    "Acute Beds": check_hospitals,
    "ICU Beds": check_hospitals,
    "Ventilators": check_hospitals,
}

red = "#F99FAB"
green = "#7ECDF7"
blue = "#8DF7C4"

stylemap = {
    "failed": f"background-color: {red};",
    "met": f"background-color: {green};",
    "exceeded": f"background-color: {blue};",
    "": "background-color: white;",
}


In [4]:
indicator_names = ["Cases", "Deaths", "Daily Testing", "Positive Tests", "Acute Beds", "ICU Beds", "Ventilators"]


county = pd.DataFrame(
    {"LA County": [county_case_indicator, 
                county_death_indicator, 
                county_test_indicator, 
                county_positive_indicator, 
                acute_indicator, 
                icu_indicator, 
                ventilator_indicator]},
    index=indicator_names
)


city = pd.DataFrame(
    {"City of LA": [city_case_indicator, 
                np.nan, 
                city_test_indicator, 
                city_positive_indicator, 
                np.nan, 
                np.nan, 
                np.nan]},
    index=indicator_names
)

In [5]:
def get_colors(df, fnmap):
    df2 =  (df.style
            .format("{:,}", na_rep="-")
            .apply(lambda row: fnmap[row.name](row).map(stylemap), axis=1)
           )
    return df2 


county1 = get_colors(county, county_fnmap)
city1 = get_colors(city, city_fnmap)

display(county1)
display(city1)

,LA County
Cases,3.0
Deaths,8.0
Daily Testing,"9,675.0"
Positive Tests,0.12
Acute Beds,0.18
ICU Beds,0.14
Ventilators,0.54


,City of LA
Cases,5.0
Deaths,-
Daily Testing,"2,774.0"
Positive Tests,0.1
Acute Beds,-
ICU Beds,-
Ventilators,-


## Original html tables

In [6]:
county_html = (county.style
               .format("{:,}", na_rep="-")
               .apply(lambda row: county_fnmap[row.name](row).map(stylemap), axis=1)
               .render()              
              )
IPython.display.HTML(county_html)


city_html = (city.style
               .format("{:,}", na_rep="-")
               .apply(lambda row: city_fnmap[row.name](row).map(stylemap), axis=1)
               .render()
            )
IPython.display.HTML(city_html)


,City of LA
Cases,5.0
Deaths,-
Daily Testing,"2,774.0"
Positive Tests,0.1
Acute Beds,-
ICU Beds,-
Ventilators,-


In [7]:
from IPython.display import display_html

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
display_side_by_side(county_html, city_html)

,LA County
Cases,3.0
Deaths,8.0
Daily Testing,"9,675.0"
Positive Tests,0.12
Acute Beds,0.18
ICU Beds,0.14
Ventilators,0.54
,City of LA
Cases,5.0
Deaths,-


In [8]:
"""
import IPython.display


# Check cases according to some criterion
def check_cases(row):
    status = ["failed" if x > 10 else "exceeded" if x < 5 else "met" for x in row]
    return pd.Series(status, index=row.index, dtype="category")

# Check deaths according to some criterion
def check_deaths(row):
    status = ["failed" if x > 5 else "exceeded" if x < 1 else "met" for x in row]
    return pd.Series(status, index=row.index, dtype="category")

# Check tests according to some criterion
def check_tests(row):
    status = ["failed" if x < 1000 else "exceeded" if x > 2000 else "met" for x in row]
    return pd.Series(status, index=row.index, dtype="category")

# Map the failed/met/exceeded state to a CSS string for the table entry
stylemap = {
    "failed": "background-color: red;",
    "met": "background-color: green;",
    "exceeded": "background-color: blue;",
}

# Map a row name to a check function
fnmap = {
    "cases": check_cases,
    "deaths": check_deaths,
    "tests": check_tests,
}

df = pandas.DataFrame(
    {"county": [100, 5, 1000], "city": [50, 0, 600]},
    index=["cases", "deaths", "tests"]
)

html = df.style.apply(lambda row: fnmap[row.name](row).map(stylemap), axis=1).render()
IPython.display.HTML(html)
"""

'\nimport IPython.display\n\n\n# Check cases according to some criterion\ndef check_cases(row):\n    status = ["failed" if x > 10 else "exceeded" if x < 5 else "met" for x in row]\n    return pd.Series(status, index=row.index, dtype="category")\n\n# Check deaths according to some criterion\ndef check_deaths(row):\n    status = ["failed" if x > 5 else "exceeded" if x < 1 else "met" for x in row]\n    return pd.Series(status, index=row.index, dtype="category")\n\n# Check tests according to some criterion\ndef check_tests(row):\n    status = ["failed" if x < 1000 else "exceeded" if x > 2000 else "met" for x in row]\n    return pd.Series(status, index=row.index, dtype="category")\n\n# Map the failed/met/exceeded state to a CSS string for the table entry\nstylemap = {\n    "failed": "background-color: red;",\n    "met": "background-color: green;",\n    "exceeded": "background-color: blue;",\n}\n\n# Map a row name to a check function\nfnmap = {\n    "cases": check_cases,\n    "deaths": check